```sql
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

This pybook is for the old flow for the file: 'https://lcps.nu/wp-content/uploads/covid-19-datafeed-oud-v1.csv'

The source code is separated into multiple sections:

1. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **DATATINO CONFIGURATIONS**

---

### Sources

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'LNAZ_HOSPITAL_BED_OCCUPANCY',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;


    -- 1.1) SET ENVIRONMENTAL VARIABLES.....
    SET @is_active = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 1
        WHEN 'acceptance' THEN 1
        ELSE 1
    END;

    SELECT TOP(1)
        @workflow_id = workflows.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
    WHERE v_workflows.[NAME] = @workflow_name;

    SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
        @id = @workflow_id, 
        @workflow_name = @workflow_name,
        @description = @workflow_description,
        @schedule = '0 13,14 * * MON-FRI',
        @active = @is_active;

    -- 2) UPSERT SOURCE(S).....
    DECLARE @source NVARCHAR(256),
            @source_id INT,
            @source_name NVARCHAR(256),
            @source_description NVARCHAR(256),
            @target_name VARCHAR(256),
            @location_type VARCHAR(50),
            @security_profile VARCHAR(50);

    -- 2.1) SET ENVIRONMENTAL VARIABLES.....
    SET @source = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 'https://lcps.nu/wp-content/uploads/covid-19-datafeed-oud-v1.csv'
        WHEN 'acceptance' THEN 'https://lcps.nu/wp-content/uploads/covid-19-datafeed-oud-v1.csv'
        ELSE 'https://lcps.nu/wp-content/uploads/covid-19-datafeed-oud-v1.csv'
    END;

    SET @location_type = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 'Web'
        WHEN 'acceptance' THEN 'Web'
        ELSE 'Web'
    END;

    SET @security_profile = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 'N/A'
        WHEN 'acceptance' THEN 'N/A'
        ELSE 'N/A'
    END;

    -- 2.2) UPSERT TABLE SOURCE(S): STAGING......
    SET @source_name = 'SOURCE_Load_Hospital_Bed_Occupancy';
    SET @source_description = 'Loads bed occupancy data into target table';
    SET @target_name = CONCAT('VWSSTAGE.', @workflow_name);

    SELECT TOP(1)
        @source_id = [ID]
    FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
    WHERE [NAME] = @source_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
        @id = @source_id,
        @source_name = @source_name,
        @description = @source_description,
        @source = @source,
        @source_columns = 'datum|IC_bezetting_covid|IC_bezetting_noncovid|kliniek_bezetting_covid|IC_opnames_covid|kliniek_opnames_covid',
        @target_columns = 'DATUM|IC_BEDDEN_COVID|IC_BEDDEN_NON_COVID|KLINIEK_BEDDEN|IC_NIEUWE_OPNAMES_COVID|KLINIEK_NIEUWE_OPNAMES_COVID|DATE_LAST_INSERTED=GETDATE',
        @target_name = @target_name,
        @source_type = 'CsvFile',
        @location_type = @location_type,
        @delimiter_type = 'Colon',
        @security_profile= @security_profile;

    -- 2) UPSERT PROCESS(ES).....
    DECLARE @process_id INT,
            @process_name NVARCHAR(256),
            @process_description NVARCHAR(256),
            @process_source_name VARCHAR(256);

    -- 2.1) UPSERT TABLE PROCESS(ES): STAGING......
    SET @process_name = 'PROCESS_Load_Hospital_Bed_Occupancy';
    SET @process_description = 'Loads bed occupancy data into target table';
    SET @process_source_name = 'SOURCE_Load_Hospital_Bed_Occupancy';

    SELECT TOP(1)
        @process_id = processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] = @process_name 
        AND v_workflows.[NAME] = @workflow_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
        @id = @process_id,
        @process_name = @process_name,
        @description = @process_description,
        @source_name = @process_source_name,
        @schedule = '* * * * *',
        @workflow_name = @workflow_name,
        @active = 1;